In [1]:
# Read in the data
import pandas as pd# Scale the data
import numpy as np

from sklearn.preprocessing import StandardScaler# Pipeline, Gridsearch, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV# Plot the confusion matrix at the end of the tutorial
#from sklearn.metrics import plot_confusion_matrix# Classification Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
from sklearn.ensemble import VotingClassifier

#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import _stop_words as stop_words
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO
from bs4 import BeautifulSoup

import string
import re
import spacy

import gensim.downloader as api

from sklearn.metrics import accuracy_score,f1_score
#import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix 
#from sklearn.metrics import classification_report
#from sklearn import metrics

import time

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/raphaelubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2023-03-17 23:30:51.774284: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 23:30:54.156734: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-17 23:30:54.156940: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-17 23:30:54.1

In [2]:
#Recuperation du dataset
df = pd.read_csv('train.csv')

In [3]:
#suppression des colonnes non-utilisées
df = df.drop(['id','keyword','location'],axis=1)

In [4]:
#Création des différentes fonction de pré-traitements
nlp = spacy.load('en_core_web_sm')

stopwords = stop_words.ENGLISH_STOP_WORDS
lemmatizer = WordNetLemmatizer()

#converting emojis
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text

#removing html code
def remove_html(text):
    soup = BeautifulSoup(text)
    text = soup.get_text()
    return text

#removing urls
def remove_urls(text):
    pattern = re.compile(r'https?://(www\.)?(\w+)(\.\w+)(/\w*)?')
    text = re.sub(pattern, "", text)
    return text

#removing mentions
def remove_mentions(text):
    pattern = re.compile(r"@\w+")
    text = re.sub(pattern, "", text)
    return text

#removing punctuations
def remove_punctuations(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), " ",text)
    return text

#Clean global with tokenizer and lemmatizer
def clean(doc):
    text_no_namedentities = []
    document = nlp(doc)
    ents = [e.text for e in document.ents]
    for item in document:
        if item.text in ents:
            pass
        else:
            text_no_namedentities.append(item.text)
    doc = (" ".join(text_no_namedentities))

    doc = doc.lower().strip()
    doc = doc.replace("</br>", " ") 
    doc = doc.replace("-", " ") 
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    doc = " ".join([token for token in doc.split() if token not in stopwords])    
    doc = "".join([lemmatizer.lemmatize(word) for word in doc])
    return doc

In [5]:
#Application des pré-traitements
df['text'] = df['text'].apply(convert_emojis)
df['text'] = df['text'].apply(remove_html)
df['text'] = df['text'].apply(remove_urls)
df['text'] = df['text'].apply(remove_mentions)
df['text'] = df['text'].apply(remove_punctuations)

df['text'] = df['text'].apply(clean)
df = df.dropna().drop_duplicates()
df = df.reset_index()

In [6]:
docs = list(df['text'])
tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_features = 20000,ngram_range=(1,2)) 
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(docs)
docs = tfidf_vectorizer_vectors.toarray()

cv = CountVectorizer(ngram_range=(1,2))
bow = cv.fit_transform(df['text']).toarray()

In [7]:
y = df['target']
X = df['text']

X_tfidf = docs
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf,y,random_state=42)

X_bow = bow
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bow,y,random_state=42)

In [8]:
w2v_model = api.load("word2vec-google-news-300")

In [9]:
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['text'])

# Priority column acts as a target variable and other columns as predictors
CountVectorizedData=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
CountVectorizedData['Priority']=df['target']

WordsVocab=CountVectorizedData.columns[:-1]

def FunctionText2Vec(inpTextData):
    # Converting the text to numeric data
    X = vectorizer.transform(inpTextData)
    CountVecData=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
    
    # Creating empty dataframe to hold sentences
    W2Vec_Data=pd.DataFrame()
    
    # Looping through each row for the data
    for i in range(CountVecData.shape[0]):

        # initiating a sentence with all zeros
        Sentence = np.zeros(300)

        # Looping thru each word in the sentence and if its present in 
        # the Word2Vec model then storing its vector
        for word in WordsVocab[CountVecData.iloc[i , :]>=1]:
            #print(word)
            if word in w2v_model.key_to_index.keys():    
                Sentence=Sentence+w2v_model[word]
        # Appending the sentence to the dataframe
        W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
    return(W2Vec_Data)

In [10]:
W2Vec_Data=FunctionText2Vec(df['text'])

In [11]:
W2Vec_Data.reset_index(inplace=True, drop=True)
W2Vec_Data['target']=CountVectorizedData['Priority']

TargetVariable=W2Vec_Data.columns[-1]
Predictors=W2Vec_Data.columns[:-1]
 
X=W2Vec_Data[Predictors].values
y=W2Vec_Data[TargetVariable].values

In [12]:
from sklearn.preprocessing import MinMaxScaler
# Choose either standardization or Normalization
# On this data Min Max Normalization is used because we need to fit Naive Bayes

# Choose between standardization and MinMAx normalization
mm=MinMaxScaler()
mm=mm.fit(X)
X=mm.transform(X)

# Split the data into training and testing set
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X, y, test_size=0.3, random_state=428)

In [13]:
lr =  LogisticRegression(random_state=42)
dt =  DecisionTreeClassifier(random_state=42)
rf =  RandomForestClassifier(random_state=42)
knn = KNeighborsClassifier()
svm = svm.SVC(random_state=42)
mnb = MultinomialNB()
gnb = GaussianNB()
bnb = BernoulliNB()

In [14]:
grids = [dt, rf, knn, lr, svm, mnb, bnb, gnb]

In [19]:
print("Training for Bag of Words data")
train_results = []
test_results = []

for classifier in grids:
    st = time.time()
    classifier.fit(X_train_bow,y_train_bow)
    et = time.time()
    train_pred = classifier.predict(X_train_bow)
    pred = classifier.predict(X_test_bow)
    train_results.append(train_pred)
    test_results.append(pred)
    print("Training F1 Score:")
    print(classifier.__class__.__name__, f1_score(y_train_bow, train_pred))
    print("Testing F1 Score:")
    print(classifier.__class__.__name__, f1_score(y_test_bow, pred))
    print("Fit time:")
    print(classifier.__class__.__name__,et-st)
    print("----------")

train_result = 0
test_result = 0

for result in train_results:
    train_result += result

for result in test_results:
    test_result += result

train_result = [int(x) for x in np.around(train_result/len(grids))]
test_result = [int(x) for x in np.around(test_result/len(grids))]

print("Training F1 Score:")
print("Voting Classifier: ", f1_score(y_train_bow, train_result))
print("Testing F1 Score:")
print("Voting Classifier :", f1_score(y_test_bow, test_result))

In [ ]:
print("Training for TFIDF data")
train_results = []
test_results = []

for classifier in grids:
    st = time.time()
    classifier.fit(X_train_tfidf,y_train_tfidf)
    et = time.time()
    train_pred = classifier.predict(X_train_tfidf)
    pred = classifier.predict(X_test_tfidf)
    train_results.append(train_pred)
    test_results.append(pred)
    print("Training F1 Score:")
    print(classifier.__class__.__name__, f1_score(y_train_tfidf, train_pred))
    print("Testing F1 Score:")
    print(classifier.__class__.__name__, f1_score(y_test_tfidf, pred))
    print("Fit time:")
    print(classifier.__class__.__name__,et-st)
    print("----------")

train_result = 0
test_result = 0

for result in train_results:
    train_result += result

for result in test_results:
    test_result += result

train_result = [int(x) for x in np.around(train_result/len(grids))]
test_result = [int(x) for x in np.around(test_result/len(grids))]

print("Training F1 Score:")
print("Voting Classifier: ", f1_score(y_train_bow, train_result))
print("Testing F1 Score:")
print("Voting Classifier :", f1_score(y_test_bow, test_result))

In [15]:
print("Training for Word2Vec data")
train_results = []
test_results = []

for classifier in grids:
    st = time.time()
    classifier.fit(X_train_w2v,y_train_w2v)
    et = time.time()
    train_pred = classifier.predict(X_train_w2v)
    pred = classifier.predict(X_test_w2v)
    train_results.append(train_pred)
    test_results.append(pred)
    print("Training F1 Score:")
    print(classifier.__class__.__name__, f1_score(y_train_w2v, train_pred))
    print("Testing F1 Score:")
    print(classifier.__class__.__name__, f1_score(y_test_w2v, pred))
    print("Fit time:")
    print(classifier.__class__.__name__,et-st)
    print("----------")

train_result = 0
test_result = 0

for result in train_results:
    train_result += result

for result in test_results:
    test_result += result

train_result = [int(x) for x in np.around(train_result/len(grids))]
test_result = [int(x) for x in np.around(test_result/len(grids))]

print("Training F1 Score:")
print("Voting Classifier: ", f1_score(y_train_w2v, train_result))
print("Testing F1 Score:")
print("Voting Classifier :", f1_score(y_test_w2v, test_result))

Training F1 Score:
DecisionTreeClassifier 0.9860537190082644
Testing F1 Score:
DecisionTreeClassifier 0.5641323273360419
Fit time:
DecisionTreeClassifier 3.1130762100219727
----------
Training F1 Score:
RandomForestClassifier 0.9861751152073732
Testing F1 Score:
RandomForestClassifier 0.6732804232804234
Fit time:
RandomForestClassifier 9.2255117893219
----------
Training F1 Score:
KNeighborsClassifier 0.7576537911301859
Testing F1 Score:
KNeighborsClassifier 0.6415868673050615
Fit time:
KNeighborsClassifier 0.002583742141723633
----------
Training F1 Score:
LogisticRegression 0.7437241379310344
Testing F1 Score:
LogisticRegression 0.7203608247422681
Fit time:
LogisticRegression 0.5158605575561523
----------
Training F1 Score:
SVC 0.8276054763900531
Testing F1 Score:
SVC 0.7193211488250653
Fit time:
SVC 4.290931701660156
----------
Training F1 Score:
MultinomialNB 0.4773755656108598
Testing F1 Score:
MultinomialNB 0.4746059544658494
Fit time:
MultinomialNB 0.06886744499206543
----------